# Endpoint Deployment

In [1]:
import boto3
import pandas as pd
import sagemaker

from sagemaker.model import ModelPackage

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
# Establish sagemaker session
sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

In [3]:
# Establish sagemaker client
boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)

In [4]:
# Get model group
MODEL_PACKAGE_GROUP = "AirDataModelGroup"

# Get all model packages in the group, sorted by creation time
model_packages = sagemaker_client.list_model_packages(
    ModelPackageGroupName=MODEL_PACKAGE_GROUP,
    SortBy="CreationTime",
    SortOrder="Descending",
)["ModelPackageSummaryList"]

if not model_packages:
    raise ValueError(f"No models found in Model Package Group: {MODEL_PACKAGE_GROUP}")

# Get the most recent model package ARN
model_package_arn = model_packages[0]["ModelPackageArn"]

print(f"Most recent model package ARN: {model_package_arn}")

Most recent model package ARN: arn:aws:sagemaker:us-east-1:768099485759:model-package/AirDataModelGroup/1


In [5]:
# Deploy endpoint
deployment_model = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
)

predictor = deployment_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
)

------!

In [6]:
# Check on deployed endpoint
response = sagemaker_client.list_endpoints()
for endpoint in response["Endpoints"]:
    print(f"Endpoint Name: {endpoint['EndpointName']}, Status: {endpoint['EndpointStatus']}")

Endpoint Name: AirDataModelGroup-2025-02-24-05-34-35-395, Status: InService


In [7]:
# Load in simulated production data
production_data = pd.read_csv("data/production_data.csv")
prod_X = production_data["value"].to_numpy().reshape(-1, 1)

In [8]:
# Simulate feeding data every 60 seconds
for i in range(len(prod_X)):
    # Send one sample at a time
    input = prod_X[i].tolist()

    # Send data to the endpoint for inference
    result = query_endpoint(predictor.endpoint_name, input)

    # Sleep for 60 seconds before sending the input
    sleep(60)

NameError: name 'query_endpoint' is not defined